# A Demonstration of Exploitation/Exploration Trade-off
## Inspired by the following notebook:
[this notebook](https://github.com/fmfn/BayesianOptimization/blob/master/examples/exploitation%20vs%20exploration.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import optimisation as op
import optimisation_gui as op_gui

In [ ]:
# Make deterministic
np.random.seed(42)

# Target Function

In [ ]:
f = lambda x: np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1)
xs = np.linspace(-2, 10, 10000)
ys = f(xs)
best_y = 1.4

class TestEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x)
evaluator = TestEvaluator()

plt.plot(xs, ys)
plt.show()

# Helper Functions
These functions set up the optimiser to act like [this library](https://github.com/fmfn/BayesianOptimization/blob/master/examples/exploitation%20vs%20exploration.ipynb)

In [ ]:
def create_optimiser(ac_func, ac_args):
    '''
    create an optimiser which is configured to be as close those used by the
    Bayesian optimisation library this notebook is copying, with the acquisition
    function and arguments for it passed as parameters
    '''
    #ranges = {'x' : xs, 'y' : [1,2,3]}
    ranges = {'x' : xs}
    gp_params = {'alpha': 1e-5, 'kernel':1.0*gp.kernels.Matern(nu=2.5), 'n_restarts_optimizer':2}
    # these settings are very over-kill, the other library uses 250 restarts which is far too many
    ac_max_params = {'num_random' : 1e5, 'num_restarts' : 10}
    
    return op.BayesianOptimisationOptimiser(
        ranges, maximise_cost=True,
        acquisition_function=ac_func, acquisition_function_params=ac_args,
        gp_params=gp_params, pre_samples=2, ac_max_params=ac_max_params, close_tolerance=1e-15)

def run_optimiser(ac_func, ac_args):
    ''' create an optimiser, run it and display the results '''
    bo = create_optimiser(ac_func, ac_args)
    bo.run_sequential(evaluator, max_jobs=25)

    def plot(n, step):
        bo.plot_step_slice('x', n, true_cost=f);
    op_gui.step_log_slider(bo, plot, pre_compute=False)
    return bo

# Upper Confidence Bound: Prefer Exploitation
$\kappa=1.0$

In [ ]:
bo = run_optimiser('UCB', {'kappa' : 1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);

# Upper Confidence Bound: Prefer Exploration
$\kappa=10.0$

In [ ]:
bo = run_optimiser('UCB', {'kappa' : 10})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);

# Expected Improvement: Prefer Exploitation

The problem with my implementation is that it does not spend as much resources as the other library does in finding the best local optimum of the acquisition function, and so towards the end the peak is too thin to land on through gradient-based optimisation. That is why this method still appears to be favouring exploration.

$\xi=0.0001$

In [ ]:
bo = run_optimiser('EI', {'xi' : 1e-4})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);

# Expected Improvement: Prefer Exploration
$\xi=0.1$

In [ ]:
bo = run_optimiser('EI', {'xi' : 0.1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);

# Probability of Improvement: Prefer Exploitation
$\xi=0.0001$

In [ ]:
bo = run_optimiser('PI', {'xi' : 1e-4})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);

# Probability of Improvement: Prefer Exploration
$\xi=0.1$

In [ ]:
bo = run_optimiser('PI', {'xi' : 0.1})

In [ ]:
bo.plot_cost_over_time(true_best=best_y);